# Qwen/Qwen2-VL-7B-Instruct 제로샷 테스트
- 4bit 양자화 + 학습 없이 바로 추론
- 테스트데이터 3887개 전부 추론 => 캐글 제출용

### [ 실행 시간 ]
- [모델 로드] 581.44s
- [추론] 총 43.24분, 샘플당 0.67초

### [ 캐글 스코어 :  ]
- Qwen/Qwen2.5-VL-3B-Instruct 와 비교할 예정



In [ ]:
# ✅ Colab 환경 준비: CUDA 12.1용 PyTorch + 필수 라이브러리 설치 후 자동 재시작
import os, time
print("필수 패키지 설치 중... ⏳")

# Hugging Face/PEFT/bitsandbytes 등
!pip -q install "transformers>=4.44.2" "accelerate>=0.34.2" "peft>=0.13.2" "bitsandbytes>=0.43.1" pillow pandas --upgrade

# ⚠️ 핵심: CUDA 12.1용 PyTorch/torchvision 설치 (CPU 빌드 방지)
!pip -q install --upgrade torch torchvision --index-url https://download.pytorch.org/whl/cu121

print("\n✅ 설치 완료. 런타임을 자동으로 재시작합니다...")
time.sleep(2)
os.kill(os.getpid(), 9)  # 자동 재시작


필수 패키지 설치 중... ⏳
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 97.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 93.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.3 which is incompatible.
gradio 5.49.1 requires pillow<12.0,>=8.0, but you have pillow 12.0.0 which is incompatible.
cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.3 which is incompatible.
dask-cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.3 which is incompatible.

✅ 설치 완료. 런타임을 자동으로 재시작합니다...


In [1]:
# ✅ GPU 사용 가능 여부/드라이버 확인
import time, torch
t0 = time.perf_counter()

# NVIDIA 드라이버/메모리 상태
!nvidia-smi

print("GPU 사용 가능 여부:", torch.cuda.is_available())
print("PyTorch CUDA 버전:", torch.version.cuda)
if torch.cuda.is_available():
    print("GPU 이름:", torch.cuda.get_device_name(0))

# ⏱️ 측정
if torch.cuda.is_available(): torch.cuda.synchronize()
print(f"[체크 소요시간] {(time.perf_counter()-t0):.3f}s")


Thu Oct 23 18:34:36 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   55C    P8             12W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# ✅ Google Drive 마운트 및 데이터 압축 해제 (한글 주석)
from google.colab import drive
import os, time

drive.mount('/content/drive')

zip_path = "/content/drive/My Drive/aichallenge/data.zip"
unzip_target = "/content/"

need_unzip = (not os.path.isdir('/content/test')) or (len(os.listdir('/content/test')) == 0)
if need_unzip:
    t0 = time.perf_counter()
    print("데이터 압축 해제 중... ⏳")
    !unzip -q "/content/drive/My Drive/aichallenge/data.zip" -d "/content/"
    print("압축 해제 완료.")
    print(f"[압축 해제 소요시간] {(time.perf_counter()-t0):.2f}s")
else:
    print("데이터가 이미 준비되어 있어 압축 해제를 건너뜁니다.")

assert os.path.exists('/content/test.csv') and os.path.exists('/content/test'), "데이터 경로 확인 필요"
print("확인 완료: /content/test.csv, /content/test/")


Mounted at /content/drive
데이터 압축 해제 중... ⏳
압축 해제 완료.
[압축 해제 소요시간] 17.99s
확인 완료: /content/test.csv, /content/test/


### ✋모델명을 바꿔서, 둘다 체크해보자
1. "Qwen/Qwen2.5-VL-7B-Instruct"
2. "Qwen/Qwen2-VL-7B-Instruct"

In [5]:
# ✅ Qwen2-VL-7B-Instruct 제로샷: 모델 로드 + 3샘플 스모크 테스트 (한글 주석 + 시간 측정)
import time, torch
from PIL import Image
import pandas as pd
from transformers import AutoModelForVision2Seq, AutoProcessor, BitsAndBytesConfig

device = "cuda" if torch.cuda.is_available() else "cpu"
print("디바이스:", device)

# 이미지 해상도: T4에서 384 권장 (크면 극단적으로 느려짐)
IMAGE_SIZE = 384
MODEL_ID = "Qwen/Qwen2-VL-7B-Instruct"

# 4bit 양자화 설정 (T4 안정 실행)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

t0 = time.perf_counter()
processor = AutoProcessor.from_pretrained(
    MODEL_ID,
    min_pixels=IMAGE_SIZE*IMAGE_SIZE,
    max_pixels=IMAGE_SIZE*IMAGE_SIZE,
    trust_remote_code=True,
)
model = AutoModelForVision2Seq.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",            # GPU가 있으면 자동으로 cuda로 올라감
    trust_remote_code=True,
)
model.eval()
if torch.cuda.is_available(): torch.cuda.synchronize()
print(f"[모델 로드] {(time.perf_counter()-t0):.2f}s")

SYSTEM_INSTRUCT = (
    "You are a helpful visual question answering assistant. "
    "Answer using exactly one letter among a, b, c, or d. No explanation."
)

def build_mc_prompt(question, a, b, c, d):
    return (
        f"{question}\n"
        f"(a) {a}\n(b) {b}\n(c) {c}\n(d) {d}\n\n"
        "정답을 반드시 a, b, c, d 중 하나의 소문자 한 글자로만 출력하세요."
    )

def extract_choice(text: str) -> str:
    text = text.strip().lower()
    lines = [l.strip() for l in text.splitlines() if l.strip()]
    if not lines: return "a"
    last = lines[-1]
    if last in ["a","b","c","d"]: return last
    for tok in last.split():
        if tok in ["a","b","c","d"]:
            return tok
    return "a"

# 🔎 스모크 테스트: 앞 3개 샘플만 돌려서 속도/정상 동작 확인
TEST_CSV = "/content/test.csv"
test_df = pd.read_csv(TEST_CSV)
assert {"id","path","question","a","b","c","d"}.issubset(test_df.columns), "test.csv 컬럼 확인 필요"

n_smoke = min(3, len(test_df))
print(f"스모크 테스트 샘플 수: {n_smoke}")

t_all = time.perf_counter()
for i in range(n_smoke):
    row = test_df.iloc[i]
    img = Image.open(row["path"]).convert("RGB")

    messages = [
        {"role": "system", "content": [{"type": "text", "text": SYSTEM_INSTRUCT}]},
        {"role": "user", "content": [
            {"type": "image", "image": img},
            {"type": "text",  "text": build_mc_prompt(row["question"], row["a"], row["b"], row["c"], row["d"])},
        ]},
    ]
    text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = processor(text=[text], images=[img], return_tensors="pt").to(device)

    t1 = time.perf_counter()
    with torch.no_grad():
        # temperature 같은 불필요 플래그 제거, max_new_tokens=2로 짧게
        out_ids = model.generate(
            **inputs,
            max_new_tokens=2,
            do_sample=False,
            eos_token_id=processor.tokenizer.eos_token_id,
        )
    if torch.cuda.is_available(): torch.cuda.synchronize()
    dt = time.perf_counter() - t1

    output_text = processor.batch_decode(out_ids, skip_special_tokens=True)[0]
    print(f"[{i}] pred={extract_choice(output_text)} | {dt:.2f}s")

t_total = time.perf_counter() - t_all
if torch.cuda.is_available(): torch.cuda.synchronize()
print(f"[스모크 총 소요] {t_total:.2f}s | 샘플당 평균 {t_total/n_smoke:.2f}s")


디바이스: cuda


preprocessor_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

model-00002-of-00005.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/3.90G [00:00<?, ?B/s]

KeyboardInterrupt: 

In [4]:
# ✅ 전체 추론 실행 + 시간 측정 + 제출 파일 저장
import time, pandas as pd
from PIL import Image
from tqdm import tqdm

TEST_CSV = "/content/test.csv"
SAVE_PATH = "/content/submission.csv"

test_df = pd.read_csv(TEST_CSV)
preds = []

t0 = time.perf_counter()
for i in tqdm(range(len(test_df)), desc="Zero-shot Inference", unit="sample"):
    row = test_df.iloc[i]
    img = Image.open(row["path"]).convert("RGB")

    messages = [
        {"role": "system", "content": [{"type": "text", "text": SYSTEM_INSTRUCT}]},
        {"role": "user", "content": [
            {"type": "image", "image": img},
            {"type": "text",  "text": build_mc_prompt(row["question"], row["a"], row["b"], row["c"], row["d"])},
        ]},
    ]
    text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = processor(text=[text], images=[img], return_tensors="pt").to(device)

    with torch.no_grad():
        out_ids = model.generate(
            **inputs,
            max_new_tokens=2,
            do_sample=False,
            eos_token_id=processor.tokenizer.eos_token_id,
        )
    output_text = processor.batch_decode(out_ids, skip_special_tokens=True)[0]
    preds.append(extract_choice(output_text))

# ⏱️ 총/평균 시간 출력
if torch.cuda.is_available(): torch.cuda.synchronize()
elapsed = time.perf_counter() - t0
print(f"[추론] 총 {elapsed/60:.2f}분, 샘플당 {elapsed/len(test_df):.2f}초")

submission = pd.DataFrame({"id": test_df["id"], "answer": preds})
submission.to_csv(SAVE_PATH, index=False)
print("Saved:", SAVE_PATH)

# 상위 5행 확인
submission.head()


Zero-shot Inference: 100%|██████████| 3887/3887 [43:14<00:00,  1.50sample/s]

[추론] 총 43.24분, 샘플당 0.67초
Saved: /content/submission.csv


,id,answer
0,test_0001,b
1,test_0002,b
2,test_0003,b
3,test_0004,c
4,test_0005,c
